# Convert ASCII Grid File to Cloud Optimised GeoTIFF

Downloads a zipped ASCII grid elevation file and exports it as a COG.

### Load packages

In [ ]:
import os
import pathlib
import rasterio.plot

from matplotlib import pyplot
from rasterio.io import MemoryFile
from rio_cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from urllib.parse import urlparse

### Settings

In [ ]:
# the URL of the ZIP file
url = "https://portal.spatial.nsw.gov.au/download/dem/56/Sydney-DEM-AHD_56_5m.zip"

# the path to save the COG image
output_path = "./data"

### Create Virtual File System URL

Rasterio can read the input image directly from a ZIP file using the Virtual File System (VFS) protocol. Here, we can use a VFS URL to download & open the file with a single line of code

VFS over HTTP sample URL: `zip+https://example.com/files.zip!/folder/file.tif`

*Note: if testing with a different ZIP file - you'll need to know what the data file's name is to use a VFS URL.*

In [ ]:
# get the name of the input file & it's extension
parsed_url = urlparse(url)
file_name, file_extension = os.path.splitext(os.path.basename(parsed_url.path))

if file_extension == ".zip":
    # format vfs url - in this case, the input .asc file shares the same name as the ZIP file 
    url = f"zip+{url}!{file_name}.asc"

### Set output file path

In [ ]:
# create required output directory
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

# COG files have a .tif extension
output_file_path = os.path.join(output_path, f"{file_name}.tif")

### Download file & convert to COG

Takes ~45 seconds on a 2018 MacBook with a 100mb/s Internet connection. Status will be displayed automatically below.

Input ASCII grid file: 790Mb; output COG file size: 240mb

In [ ]:
%%time
# create a COG profile to set compression on the output file
dst_profile = cog_profiles.get("deflate")

# download and open the input file
with rasterio.open(url) as input_image:
    # Create the COG in-memory & save it locally - in-memory is useful if you want to save it remotely (e.g. AWS S3)
    with MemoryFile() as output_image:
        cog_translate(input_image, output_image.name, dst_profile, in_memory=True, nodata=-9999)

        # save the COG to disk
        with open(output_file_path, "wb") as f:
            f.write(output_image.read())

### Open & display the COG file

Takes ~10 seconds

In [ ]:
%%time
with rasterio.open(output_file_path) as dataset:
    fig = pyplot.figure(figsize=(9, 12), dpi=80)
    rasterio.plot.show(dataset, cmap="terrain")